# Explaining the fast CBC algorithm

In [1]:
from test import tools
import numpy as np

In [3]:
d = 5
n = 7
omega = lambda x: np.pi*np.pi*(x**2-x+1/6)
omega_n = omega(np.arange(0,n)/n)
gamma = np.power(np.arange(1,d+1),-1.5) 

Assume that some vector $z_0 \in \mathbb{Z}_N^{d-1}$ is optimized until $d-1$ dimensions.

In [9]:
z0 = np.random.randint(1, n, d-1)
z0

array([5, 4, 3, 2])

Goal: Optimize the following worst case error function

$$ e_{s}^{2}\left(z_{s}\right)=-1+\frac{1}{n} \sum_{k=0}^{n-1} \prod_{j=1}^{s}\left(1+\gamma_{j} \omega\left(\left\{\frac{k z_{j}}{n}\right\}\right)\right) . $$

Important step is to break the product above into two parts:

$$ \begin{aligned} \prod_{j=1}^{s}\left(1+\gamma_{j} \omega\left(\left\{\frac{k z_{j}}{n}\right\}\right)\right)=& \left(1+\gamma_{s} \omega\left(\left\{\frac{k z_{s}}{n}\right\}\right)\right) \\ & \cdot  \left(\prod_{j=1}^{s-1}\left(1+\gamma_{j} \omega\left(\left\{\frac{k z_{j}}{n}\right\}\right)\right)\right) \end{aligned} . $$

And then:

$$ e_{s}^{2}\left(z_{s}\right)=-1+\frac{1}{n} \sum_{k=0}^{n-1}\left(1+\gamma_{s} \omega\left(\left\{\frac{k z_{s}}{n}\right\}\right)\right) \cdot \mathbf{p}_{s-1}[k] . $$

The vector $ \mathbf{p}_{s-1} \in \mathbb{R}^n$ :

$$ \mathbf{p}_{s-1}[k] =  \prod_{j=1}^{s-1}\left(1+\gamma_{j} \omega\left(\left\{\frac{k z_{j}}{n}\right\}\right)\right) $$ .


In [10]:
p = np.ones(n)
for j in np.arange(0,d-1):
    p = np.multiply(p, 1 + gamma[j]*omega_n[z0[j]*np.arange(0,n) % n])

Now, compute at each step:
$$
\begin{aligned} \mathbf{e}_{s}^{2} &=-1+\frac{1}{n} K_{n, \gamma_{s}} \cdot \mathbf{p}_{s-1} \\ &=-1+\frac{1}{n}\left(1+\gamma_{s} \Omega_{n}\right) \cdot \mathbf{p}_{s-1} \end{aligned}
$$


Rader factorization

$$
\Omega_{n}=\Pi_{n, \rho}^{\prime} \cdot\left[\begin{array}{cc}\omega_{0} \mathbf{1}_{n-1} & C_{n-1}\end{array}\right] \cdot \Pi_{n, \rho^{-1}}^{T}
$$,

where $\rho$ is a primitive root of $n$, and
$$
\begin{aligned} C_{n-1} &=\operatorname{circ}\left(\boldsymbol{\psi}^{\prime}\right) \\ \boldsymbol{\psi} &:=\Pi_{n, \rho}^{T} \cdot \boldsymbol{\omega} \end{aligned}
$$.

In [11]:
from test.generatorp import generatorp

In [12]:
g = generatorp(7)
g

3

Representation of the permutation matrix
$ \Pi_{n, \rho}^{T} $ , without the first row and column:

In [14]:
perm = np.zeros(n-1, dtype=int)
perm[0] = 1
for i in np.arange(1, n-1):
    perm[i] = (perm[i-1]*g) % n
perm

array([1, 3, 2, 6, 4, 5])

The vector $\boldsymbol{\psi}^{\prime}$ :

In [15]:
psi0 = omega(0)
psi = omega(np.divide(perm,n))
psi

array([ 0.43641108, -0.77211191, -0.36927091,  0.43641108, -0.77211191,
       -0.36927091])

In [16]:
fft_psi = np.fft.fft(psi)
fft_psi

array([-1.40994349e+00+0.j        ,  2.77555756e-16+0.j        ,
        2.01420498e+00+0.69774107j,  2.77555756e-16+0.j        ,
        2.01420498e+00-0.69774107j,  2.77555756e-16+0.j        ])

Create $\Pi_{n, \rho^{-1}}^{T}$ :

In [10]:
# perm_inv = 
perm_inv = np.zeros(n-1, dtype=int)
for i in np.arange(1, n):
    perm_inv[perm[i-1]-1]= i
perm_inv

Error $\mathbf{e}_{s}^{2}$

In [11]:
# e2 = np.fft.ifft(np.multiply(fft_psi, np.fft.fft(p[perm_inv])))